In [ ]:
import finnhub
import random
import time
import json
from confluent_kafka import Producer

# ================= CONFIGURATION =================
FINNHUB_API_KEY = "d3ipp99r01qq7dnem780d3ipp99r01qq7dnem78g"
KAFKA_BROKER = "3.133.95.1:9092"  # your Kafka broker address
KAFKA_TOPIC = "finnhub_profiles"

# Initialize Finnhub client
finnhub_client = finnhub.Client(api_key=FINNHUB_API_KEY)

# Example list of 100+ known stock symbols
symbols_list = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "META", "TSLA", "NFLX", "NVDA", "ORCL", "INTC",
    # ... (rest of the symbols)
]

# Initialize Kafka producer
producer = Producer({"bootstrap.servers": KAFKA_BROKER})

def delivery_report(err, msg):
    if err:
        print(f"❌ Message failed delivery: {err}")
    else:
        print(f"✅ Message delivered to {msg.topic()} [{msg.partition()}]")

# Fetch company profiles and send to Kafka
def send_profiles_to_kafka(num_companies=50):
    selected_symbols = random.sample(symbols_list, num_companies)
    for symbol in selected_symbols:
        try:
            profile = finnhub_client.company_profile2(symbol=symbol)
            data = {
                "symbol": symbol,
                "name": profile.get("name", ""),
                "exchange": profile.get("exchange", ""),
                "industry": profile.get("finnhubIndustry", ""),
                "website": profile.get("weburl", "")
            }
            producer.produce(KAFKA_TOPIC, key=symbol, value=json.dumps(data), callback=delivery_report)
            producer.flush()
            time.sleep(0.5)  # avoid hitting API rate limits
        except Exception as e:
            print(f"Error fetching profile for {symbol}: {e}")

if __name__ == "__main__":
    send_profiles_to_kafka(num_companies=50)


In [21]:
import finnhub
import random
import csv

# Initialize Finnhub client
finnhub_client = finnhub.Client(api_key="d3ipp99r01qq7dnem780d3ipp99r01qq7dnem78g")

# List of company symbols
companies = [
    'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'FB', 'TSLA', 'NVDA', 'JPM', 'V', 'JNJ',
    'WMT', 'DIS', 'NFLX', 'ADBE', 'PYPL', 'INTC', 'CSCO', 'ORCL', 'KO', 'PFE',
    'NKE', 'BA', 'MCD', 'CRM', 'PEP', 'ABBV', 'ABT', 'T', 'XOM', 'CVX'
]

# Pick 20 random companies
random_companies = random.sample(companies, 20)

# CSV file name
csv_file = "company_profiles.csv"

# Define the CSV headers
headers = ["symbol", "name", "country", "industry", "weburl", "exchange", "ipo", "marketCapitalization"]

# Open CSV file to write
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=headers)
    writer.writeheader()
    
    for symbol in random_companies:
        profile = finnhub_client.company_profile2(symbol=symbol)
        
        # Write a row with selected fields
        writer.writerow({
            "symbol": profile.get("ticker", ""),
            "name": profile.get("name", ""),
            "country": profile.get("country", ""),
            "industry": profile.get("finnhubIndustry", ""),
            "weburl": profile.get("weburl", ""),
            "exchange": profile.get("exchange", ""),
            "ipo": profile.get("ipo", ""),
            "marketCapitalization": profile.get("marketCapitalization", "")
        })

print(f"CSV file '{csv_file}' created successfully!")


CSV file 'company_profiles.csv' created successfully!
